In [179]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import torch

In [180]:
# Load CSV into a Pandas DataFrame
df = pd.read_csv('profiledescriptions_withpartyanduserid.csv', sep=';')

In [181]:
df.head()

,X,twitter_handle,profile,user_id,party
0,0,ManjaSchuele,"Ministerin für Wissenschaft, Forschung und Kul...","8,27091E+17",SPD
1,1,TeamPetraPau,hier twittert das Team von Petra Pau/Marzahn-H...,1683845126,DIE LINKE
2,2,TeamDieSchmidt,"Hier tweetet, textet und publiziert das Team d...",1377117206,SPD
3,3,reinhardbrandl,Mitglied im Verteidigungsausschuss 🇺🇦 \nMitgli...,262730721,CSU
4,4,schwarz_spd,SPD Bundestagskandidat für Bamberg und Forchhe...,53617577,SPD


In [182]:
df.shape

(2049, 5)

In [183]:
df_test = pd.read_csv('labeled_testdata_bert.csv')

In [184]:
df_test.head()

,X,twitter_handle,example,user_id,party,label
0,1789,ABaerbock,Offizieller Twitter Account von Außenministeri...,1.269262e+18,Bündnis 90/Die Grünen,1
1,190,ABaerbockArchiv,Dies ist ein Archiv des Twitter-Accounts von A...,2.179011e+09,Bündnis 90/Die Grünen,1
2,920,ACvA_MdL,"Landtagsabgeordnete der @cdufraktionmv, stellv...",1.218517e+18,CDU,3
3,822,ADressel,Finanzsenator der Freien und Hansestadt Hambur...,1.053199e+18,SPD,2
4,1202,AG_AfD,Vorstehender der Bundestagsfraktion der Altern...,9.126128e+17,AfD,5


In [185]:
df_test.shape

(200, 6)

In [186]:
df = df[~df['X'].isin(df_test['X'])]

In [187]:
df.head()

,X,twitter_handle,profile,user_id,party
0,0,ManjaSchuele,"Ministerin für Wissenschaft, Forschung und Kul...","8,27091E+17",SPD
1,1,TeamPetraPau,hier twittert das Team von Petra Pau/Marzahn-H...,1683845126,DIE LINKE
2,2,TeamDieSchmidt,"Hier tweetet, textet und publiziert das Team d...",1377117206,SPD
3,3,reinhardbrandl,Mitglied im Verteidigungsausschuss 🇺🇦 \nMitgli...,262730721,CSU
4,4,schwarz_spd,SPD Bundestagskandidat für Bamberg und Forchhe...,53617577,SPD


In [188]:
df.shape

(1860, 5)

In [189]:
# Get unique values in a specific column

def analyze_party_distribution(column):
    # Get unique values in a specific column
    unique_values = column.unique()
    print("Unique Parties:", unique_values)

    # Get count of each unique value in the column
    value_counts = column.value_counts()
    print("\nValue counts:\n", value_counts)
 
analyze_party_distribution(df['party'])

Unique Parties: ['SPD' 'DIE LINKE' 'CSU' 'AfD' 'Bündnis 90/Die Grünen' 'FDP' 'CDU'
 'Liberal-Konservative Reformer' 'Christliche Mitte'
 'Bürgerbewegung pro NRW' 'Bürgerrechtsbewegung Solidarität' 'ÖDP'
 'DIE PARTEI' 'Bayernpartei' 'Sozialistische Gleichheitspartei'
 'Piratenpartei' 'no party affiliation' 'Renew Europe'
 'Europäische Volkspartei' 'Freie Wähler' 'BIW' 'LKR' 'Die Föderalen'
 'SSW' 'Thüringer Heimatpartei' 'buendnis21' 'UNABHAENGIGE Partei'
 'Klimaliste Baden-Württemberg' 'dieBasis' 'Wir2020'
 'Deutsche Kommunistische Partei DKP' 'PdF' 'Volt Deutschland' 'MLPD'
 'Die Urbane' 'Demokratie in Bewegung' 'Partei für Gesundheitsforschung'
 'V-Partei³' 'Graue Panther' 'Bündnis C' 'Team Todenhöfer'
 'Partei der Humanisten' 'MENSCHLICHE WELT' 'Tierschutzpartei'
 'Dt. Konservative' 'Liberale Demokraten' 'FW' nan]

Value counts:
 party
SPD                                   417
Bündnis 90/Die Grünen                 380
CDU                                   314
FDP                    

In [190]:
party2id = {
    "DIE LINKE": 0,
    "Bündnis 90/Die Grünen": 1,
    "SPD": 2,
    "CSU": 3,
    "CDU": 3,
    "FDP": 4,
    "AfD": 5,
    "Keine Kategorie": 6
}

### Just keep the party names from above and change any other party to "Keine Kategorie"

In [191]:
# List of valid party names
valid_parties = ["DIE LINKE", "Bündnis 90/Die Grünen", "SPD", "CSU", "CDU", "FDP", "AfD"]


# Function to map party names
def map_parties(party):
    if party in valid_parties:
        return party
    else:
        return "Keine Kategorie"

# Apply the function to the 'Party' column
df.loc[:,'party'] = df['party'].apply(map_parties)

In [192]:
analyze_party_distribution(df['party'])

Unique Parties: ['SPD' 'DIE LINKE' 'CSU' 'AfD' 'Bündnis 90/Die Grünen' 'FDP' 'CDU'
 'Keine Kategorie']

Value counts:
 party
SPD                      417
Bündnis 90/Die Grünen    380
CDU                      314
FDP                      199
AfD                      190
DIE LINKE                165
Keine Kategorie          127
CSU                       68
Name: count, dtype: int64


### Map partys to label

In [193]:
df.loc[:,"label"] = df["party"].map(party2id.get)

In [194]:
# just to be sure that theres no na value and every value is an integer
# drop everything except of profile description and label
df = df[df['label'].notna()]
df['label'] = df['label'].astype(int)
df = df[['profile', 'label']]

In [195]:
df.head()

,profile,label
0,"Ministerin für Wissenschaft, Forschung und Kul...",2
1,hier twittert das Team von Petra Pau/Marzahn-H...,0
2,"Hier tweetet, textet und publiziert das Team d...",2
3,Mitglied im Verteidigungsausschuss 🇺🇦 \nMitgli...,3
4,SPD Bundestagskandidat für Bamberg und Forchhe...,2


#### Converting emojis to text using pip package emoji

In [196]:
import emoji
import string
import re

def demojize(text):
    # Convert emoji to text
    processed_text = emoji.demojize(text, language='de')
    # Remove ':' from converting the emojis to text
    for char in string.punctuation.replace('.', ''):
        processed_text = processed_text.replace(char, ' ')
    # Remove trailing whitespace
    processed_text = re.sub(r'\s+', ' ', processed_text).strip()
    return processed_text

df['profile'] = df['profile'].apply(lambda x: demojize(x))

In [197]:
df['profile'][1]

'hier twittert das Team von Petra Pau Marzahn Hellersdorf mahe PAUer für die LINKE'

In [198]:
# Load the BERT tokenizer
pretrained_LM = "bert-base-german-cased"
#pretrained_LM = "distilbert-base-german-cased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_LM)

def tokenize_function(examples):
    return tokenizer(examples["profile"], padding="max_length", truncation=True, return_tensors="pt")

In [199]:
# Split into training and testing data
training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)

In [200]:
# Load datasets
training_data = Dataset.from_pandas(training_data)
testing_data = Dataset.from_pandas(testing_data)

tokenized_training_data = training_data.map(tokenize_function, batched=True)
tokenized_testing_data = testing_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/1488 [00:00<?, ? examples/s]

Map:   0%|          | 0/372 [00:00<?, ? examples/s]

In [201]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-german-cased', vocab_size=30000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [206]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Good link about which average to choose: https://simonhessner.de/why-are-precision-recall-and-f1-score-equal-when-using-micro-averaging-in-a-multi-class-problem/
# micro: Calculate metrics globally by counting the total true positives, false negatives and false positives.
# macro: Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [207]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Model and training setup
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_LM, num_labels=7
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [208]:
training_args = TrainingArguments(
    output_dir="bert_full",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    warmup_steps=150,               # number of warmup steps for learning rate 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    #label_names=["LABEL"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_training_data,
    eval_dataset=tokenized_testing_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  
)

#checkpointing
#use cuda
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.697989,0.311828,0.161186,0.134388,0.208760
2,No log,0.696617,0.787634,0.778550,0.816008,0.757116
3,No log,0.565831,0.793011,0.782183,0.810816,0.766041
4,No log,0.627799,0.806452,0.802723,0.825783,0.786874
5,No log,0.698855,0.806452,0.795015,0.805247,0.787275
6,0.746400,0.848561,0.782258,0.767138,0.767166,0.771624
7,0.746400,0.883025,0.801075,0.784735,0.792222,0.780182
8,0.746400,0.950521,0.798387,0.782955,0.790716,0.778397
9,0.746400,0.965954,0.803763,0.790196,0.797451,0.785115
10,0.746400,0.982840,0.795699,0.780317,0.787761,0.775393


/software/jupyter/conda/envs/jupyter/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=930, training_loss=0.41288542696224745, metrics={'train_runtime': 675.349, 'train_samples_per_second': 22.033, 'train_steps_per_second': 1.377, 'total_flos': 3915268263936000.0, 'train_loss': 0.41288542696224745, 'epoch': 10.0})

## Evaluation on labelled test data set, 200 profile descriptions

In [209]:
# Load CSV into a Pandas DataFrame
df_eval = pd.read_csv('labeled_testdata_bert.csv')
df_eval = df_eval[['example', 'label']]
df_eval = df_eval[df_eval['label'].notna()]
df_eval['label'] = df_eval['label'].astype(int)
# Rename the 'example' column to 'profile'
df_eval.rename(columns={'example': 'profile'}, inplace=True)

In [210]:
df_eval

,profile,label
0,Offizieller Twitter Account von Außenministeri...,1
1,Dies ist ein Archiv des Twitter-Accounts von A...,1
2,"Landtagsabgeordnete der @cdufraktionmv, stellv...",3
3,Finanzsenator der Freien und Hansestadt Hambur...,2
4,Vorstehender der Bundestagsfraktion der Altern...,5
...,...,...
195,Sprecherin für Familie und Bildung der @gruene...,1
196,stellv. Vorsitzender Eisenbahn- und Verkehrsge...,2
197,Mitglied des Europäischen Parlaments,6
198,"#MdL, Parlamentarischer Geschäftsführer und Sp...",1


In [211]:
df_eval['profile'] = df_eval['profile'].apply(lambda x: demojize(x))
eval_data = Dataset.from_pandas(df_eval)
tokenized_eval_data = eval_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [212]:
# put in testing mode (dropout modules are deactivated)
model.eval() 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [213]:
trainer.predict(tokenized_eval_data)

PredictionOutput(predictions=array([[-0.61910313,  0.03925972,  0.47792742, ..., -0.9226824 ,
        -0.86336315,  2.4679937 ],
       [-0.66054785, -0.388963  ,  0.19457549, ..., -0.41623634,
        -0.6889452 ,  2.4667041 ],
       [-1.1016378 , -0.7066047 , -0.74150217, ..., -0.9151975 ,
        -0.28603762, -0.5311623 ],
       ...,
       [ 0.12678424, -1.1716827 , -0.7052942 , ..., -0.8502929 ,
         0.32220066,  1.7733136 ],
       [-0.4851758 ,  4.390679  , -0.14872812, ..., -0.60208064,
        -2.5544739 , -0.51748955],
       [-0.9979751 , -1.0155394 , -0.8772448 , ..., -0.96546763,
        -0.13496706, -0.03060766]], dtype=float32), label_ids=array([1, 1, 3, 2, 5, 2, 4, 2, 6, 2, 3, 6, 3, 3, 4, 6, 6, 2, 6, 5, 5, 6,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 3, 2, 3, 6, 1, 4, 5, 6, 3, 6, 6, 1,
       1, 6, 4, 6, 5, 3, 2, 2, 0, 5, 1, 4, 3, 6, 5, 3, 2, 3, 3, 6, 0, 6,
       5, 6, 1, 4, 2, 3, 0, 6, 6, 6, 3, 2, 6, 2, 1, 1, 1, 4, 6, 3, 1, 2,
       3, 0, 1, 5, 6, 4, 6, 6, 1, 5, 2, 